<a href="https://colab.research.google.com/github/Madhu-712/Gemini_Langchainagents/blob/main/Flightagents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [ ]:
pip install -qU langchain-google-vertexai

#langchain Agents

In [ ]:

from datetime import datetime, timedelta
import pytz
import os
import requests
from langchain.agents import AgentExecutor, create_react_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool
from langchain.prompts import PromptTemplate
from langchain.google_serp import GoogleSerpAPIWrapper

#Set up Api key for aerodat
AERODAT_BASE_URL = "https://aerodat.flightaware.com/aerodat"
AERODAT_KEY = os.getenv("AERODAT_KEY")

#functions
def get_api_session():
    session = requests.Session()
    session.headers.update({"x-apikey": AERODAT_KEY})
    return session

def fetch_flight_data(flight_id, session):
    if " " in flight_id:
        flight_id = flight_id.split(" ")[1]

    start_date = (datetime.now().date() - timedelta(days=1)).strftime("%Y-%m-%d")
    end_date = datetime.now().date().strftime("%Y-%m-%d")

    api_resource = f"/flights?flightId={flight_id}&startDate={start_date}&endDate={end_date}"

    response = session.get(AERODAT_BASE_URL + api_resource)
    response.raise_for_status() # Raises HTTPError, if one occurred
    return response.json()['flights'][0]


def utc_to_local(utc_datetime_str, local_timezone_str):
    local_timezone = pytz.timezone(local_timezone_str)
    utc_datetime = datetime.strptime(utc_datetime_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.utc)
    local_datetime = utc_datetime.astimezone(local_timezone)
    return local_datetime.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:

# Ensure your VertexAI credentials are configured

from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-1.5-flash")

In [ ]:
@tool
def get_flight_status(flight_id:str):
     """Returns Flight Information."""
      session = get_api_session()
        flight_data = fetch_flight_data(flight_id, session)


        dep_key = "estimatedOut" if "estimatedOut" in flight_data and flight_data["estimatedOut"] else \
                   "actualOut" if "actualOut" in flight_data and flight_data["actualOut"] else \
                  "scheduledOut"


        arr_key = "estimatedIn" if "estimatedIn" in flight_data and flight_data["estimatedIn"] else \
                   "actualIn" if "actualIn" in flight_data and flight_data["actualIn"] else \
                  "scheduledIn"

        flight_details = {
            "source": flight_data["origin"]["city"],
            "destination" : flight_data["destination"]["city"],
            "depart_time": utc_to_local(flight_data.get(dep_key), flight_data["origin"]["timezone"]),
            "arrival_time": utc_to_local(flight_data.get(arr_key), flight_data["destination"]["timezone"]),
            "status": flight_data["status"]
        }


        return f"""The current status of flight {flight_id} from {flight_details["source"]} to {flight_details["destination"]} is {flight_details["status"]} with the departure time as {flight_details["depart_time"]} and arrival time as {flight_details["arrival_time"]}"""

In [ ]:
tools =GetFlightStatus()

In [ ]:
prompt = PromptTemplate(
    template="""Answer the following questions as best as you can. You have access to the following tools:{tool_names}

Use the following format:

```tool_code
{agent_scratchpad}```""",
    input_variables=["input", "tool_names", "agent_scratchpad"],
)

In [ ]:
prompt

In [ ]:
llm = ChatVertexAI(model="gemini-1.5-flash")

In [ ]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
Output=agent_executor.invoke({"input":"what is the status of EK24 and also include source and destination along with the timings"},verbose=True)

In [ ]:
response(Output)

In [ ]:
Output=agent_executor.invoke({"input":"If I am arriving Hyderabad by EK24 and it takes about 1hr to get luggages and getting home takes 2 hrs from airport. What time I should book a cab and when will I reach Home?},verbose=True)

In [ ]:
response(Output)